Import packages 

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics.pairwise import euclidean_distances
from scipy.linalg import eigh
from scipy.spatial import distance

Read fish csv

In [2]:
fish_df = pd.read_csv('fish_data.csv')


Q5(a)

Train test split 

In [3]:
X_train = fish_df.iloc[:60, 1:3]
y_train = fish_df.iloc[:60, -1]
X_test = fish_df.iloc[60:, 1:3]
y_test = fish_df.iloc[60:, -1]


Fit a linear regression model to the training data

In [4]:
reg = LinearRegression().fit(X_train, y_train)

Extract the coefficients of the linear regression model

In [5]:
b0 = reg.intercept_ # Y intercept
b1, b2 = reg.coef_ #coefficiants of x1,x2

Compute the regression function with the computed parameters (Yi = B0 + B1X1 + B2X2)

In [6]:
y_train_pred = b0 + b1*X_train.iloc[:, 0] + b2*X_train.iloc[:, 1] 

Compute the empirical risk Rempf/mse for the linear regression model

In [7]:
Rempf = ((y_train_pred - y_train)**2).mean()

In [8]:
Rempf_sk = mse(y_train, y_train_pred)

Q5(a) = Regression function and the empirical risk

In [9]:

print("Regression function: y = {:.2f} + {:.2f}*x1 + {:.2f}*x2 ".format(b0, b1, b2))
print("Empirical risk Rempf: {:.2f}".format(Rempf))

Regression function: y = 2988.86 + 22.53*x1 + -87.02*x2 
Empirical risk Rempf: 1501627.68


Q5(b)

Given the feature map phi([x1 x2]) = [1 x1 x2 x1*x2] we nee to perform a transformation of each data point in x1 and x2 into a 4d feature vector [1 x1 x2 x1*x2]

This function goes through the 2d vector [x1 x2] to 4d vector [1 x1 x2 x1x2]

In [10]:
phi_df = fish_df.filter(['Type of fish', 'Age of fish (days)', 'Tempreture (Celsius)', 'Length (cm)' ],axis=1)

In [11]:
phi_df['Engineered feature'] = phi_df['Age of fish (days)']*phi_df['Tempreture (Celsius)']

In [12]:
# Split the data into training and test sets
phi_X_train = phi_df.iloc[:60, [1,2,4]]
phi_y_train = phi_df.iloc[:60, [3]]
phi_X_test = phi_df.iloc[60:, [1,2,4]]
phi_y_test = phi_df.iloc[60:, [3]]

changing the dependant variable from a df to a series to work when fitting the model

In [13]:
phi_y_train_ = phi_y_train.squeeze()
phi_y_test_ = phi_y_test.squeeze()

fit model

In [14]:
phi_reg = LinearRegression().fit(phi_X_train, phi_y_train_)

get the Y intercept and coefficiants 

In [15]:
v0 = phi_reg.intercept_
v1, v2, v3 = phi_reg.coef_


Compute the regression function with the computed parameters (Yi = v0 + v1X1 + v2X2 + v3X3)

In [16]:
phi_y_train_pred = v0 + v1*phi_X_train.iloc[:, 0] + v2*phi_X_train.iloc[:, 1] + v3*phi_X_train.iloc[:, 2]

Compute the empirical risk Rempf/mse for the linear regression model

In [17]:
phi_Rempf = ((phi_y_train_pred - phi_y_train_)**2).mean()

In [18]:
phi_Rempf_sk = mse(phi_y_train_, phi_y_train_pred)

In [19]:
print("Regression function: y = {:.2f} + {:.2f}*x1 + {:.2f}*x2 + {:.2f}*x3 ".format(v0, v1, v2, v3))
print("Empirical risk Rempf: {:.2f}".format(phi_Rempf))

Regression function: y = 1292.33 + 45.36*x1 + -26.33*x2 + -0.82*x3 
Empirical risk Rempf: 1496226.22


Q5(c)

Test data 

In [20]:
y_prediction = reg.predict(X_test)
test_Rempf = mse(y_test, y_prediction)

In [21]:
phi_y_prediction = phi_reg.predict(phi_X_test)
test_phi_Rempf = mse(phi_y_test_, phi_y_prediction)

Q5(d) Cannot verify if all the information is there, not done

MAP Maximum a Posteriori

formula 

logp(θ|XY) = logp(Y|X,θ) + logp(θ) +  constant.

Assumptions

σ^2 = Rempf

θa = b1 b2

Q6(a)

i.

For the 2nd 3rd and 4th row calculate the pairwise distances between all data points

In [22]:
x = fish_df.iloc[:, 1:4]

euc_distances = x.apply(lambda row: np.sqrt(((row - x)**2).sum(axis=1)), axis=1)

Find the pair with the maximum distance

In [23]:

max_pair = np.unravel_index(np.argmax(euc_distances), euc_distances.shape)
xa, xb = max_pair


xa += 1
xb += 1

In [24]:
print("Row {} and Row {} have a distance of {:.2f}".format(xa, xb, euc_distances.loc[xa-1, xb-1]))

Row 18 and Row 36 have a distance of 4181.50


In [25]:
filename = 'max_euclidean.csv'
euc_distances.to_csv(filename, index=False)

ii.

Calculate euc distances of every point from xa and xb

In [26]:
dist_a = euclidean_distances(x.iloc[:60], x.iloc[[xa]])
dist_b = euclidean_distances(x.iloc[:60], x.iloc[[xb]])

Classify data points into La and Lb

In [37]:
La = [i for i in range(60) if dist_a[i] < dist_b[i]]
Lb = [i for i in range(60) if dist_a[i] > dist_b[i]]

iii.

In [28]:
La_data = x.iloc[La]
Lb_data = x.iloc[Lb]

Compute empirical mean

In [29]:
em_mean_a = np.mean(La_data, axis=0)
em_mean_b = np.mean(Lb_data, axis=0)

Compute covariance 

In [30]:
cov_a = np.cov(La_data.T)
cov_b = np.cov(Lb_data.T)

In [31]:
print("La, Emperical mean:\n", em_mean_a)
print("La, Emperical Covariance :\n", cov_a)
print("Lb, Emperical mean:\n", em_mean_b)
print("Lb, Emperical Covariance :\n", cov_b)

La, Emperical mean:
 Age of fish (days)        88.227273
Tempreture (Celsius)      27.818182
Length (cm)             2933.545455
dtype: float64
La, Emperical Covariance :
 [[ 1.77980761e+03 -2.29809725e+01  3.04002220e+04]
 [-2.29809725e+01  5.08245243e+00 -9.26363636e+02]
 [ 3.04002220e+04 -9.26363636e+02  1.95062351e+06]]
Lb, Emperical mean:
 Age of fish (days)       48.2500
Tempreture (Celsius)     28.2500
Length (cm)             579.0625
dtype: float64
Lb, Emperical Covariance :
 [[ 4.78066667e+02 -6.66666667e-02  1.89166667e+01]
 [-6.66666667e-02  4.73333333e+00 -3.14166667e+01]
 [ 1.89166667e+01 -3.14166667e+01  3.93739583e+03]]


iv.

Verification of positive definite emperical Cov_a and Cov_b

In [32]:
eigenvals_a = eigh(cov_a, eigvals_only=True)
eigenvals_b = eigh(cov_b, eigvals_only=True)


In [33]:
eigenvals_b

array([   4.48235752,  477.963251  , 3937.75022482])

In [34]:
eigenvals_b

array([   4.48235752,  477.963251  , 3937.75022482])

v. Not done

scipy.spatial.distance.mahalanobis(u, v, VI)

In [35]:
cov_a_inv = np.linalg.inv(cov_a)
cov_b_inv = np.linalg.inv(cov_b)

In [36]:
maha_a = distance.mahalanobis(La, ? ,cov_a_inv)
maha_b = distance.mahalanobis(Lb,? ,cov_b_inv )

SyntaxError: invalid syntax (2824883085.py, line 1)

In [ ]:
maha_dist_a = np.array(maha_a)

vi. Not Done

Q6(b)

Logarithmic Regression, categorical dependant variable

In [ ]:
b,  

In [ ]:
#del m2_X_train
#del m2_y_train
#del m2_X_test 
#del m2_y_test 
#del b3
#del v0
#del b3